# Finding and Replacing Missing Data

In [ ]:
import pandas as pd
import numpy as np
import os 

## Load and Inspect the Data

In [ ]:
filename = os.path.join(os.getcwd(), "data", "adult.data.partial.missing")
df = pd.read_csv(filename, header=0)

In [ ]:
df.shape

In [ ]:
df.head()


## Dealing with Missing Data

Our goal will be to identify which columns in a dataset have missing values, and to replace a missing value in a column with the mean of the other values in that column. We will add dummy variables to our dataset to indicate which  columns initally had missing values. 

### Step 1:  Identify Missing Values Using Pandas `isnull()` Method

First let us check if there are missing values in DataFrame `df`.

In [ ]:
df.isnull().values.any()

DataFrame `df` contains missing values! The Pandas `isnull()` method returns `True`/`False` values indicating whether a value is or is not missing in a particular position in a DataFrame or Series. This method recognizes various spellings of missingness like `NaN`, `nan`, `None`, and `NA` among others.<br> Consult the online [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isnull.html) for more information.

In [ ]:
df.isnull().head()

The code cell below counts the number of times a missing value occurs in each column. It applies the `isnull()` method and then aggregates the results by columns using the `np.sum()` function. For more information about `np.sum()`, consult the online [documentation](https://numpy.org/doc/stable/reference/generated/numpy.sum.html).

In [ ]:
nan_count = np.sum(df.isnull(), axis = 0)
nan_count

The code cell below stores the names of the columns with detected missing values into a Python list.

In [ ]:
condition = nan_count != 0 # look for all columns with missing values

col_names = nan_count[condition].index # get the column names
print(col_names)

nan_cols = list(col_names) # convert column names to list
print(nan_cols)

### Step 2: Choose Which Values to Fill 

We can see that five columns in our DataFrame contain missing values. Would you want to replace the missing values with something for every one of these columns? 

Let's take a look at the data types of the columns that contain missing values using `dtypes`.

In [ ]:
nan_col_types = df[nan_cols].dtypes
nan_col_types

For three of the five identified columns, the type is 'object'. Is this a problem?<br>
    A common approach to dealing with the missing values is to replace those values with either the mean, the median, or some other type of 'representative' value wherever a `nan` occurs. This, of course, assumes that the column is numerical to begin with. That doesn't seem to be true for the `workclass`, `occupation`, and `native-country` variables.
Let us confirm:

In [ ]:
print(df['workclass'].unique())
print(df['occupation'].unique())
print(df['native-country'].unique())

The concept of 'mean' is not defined for string entries, so filling in the missing values with the mean of the column wouldn't work here. In real business settings, one way to go about filling in the missing values would be to fit a model that predicts the country based on other values. All data filling methods come with caveats, and some may threaten the validity of your larger analytical conclusions.

For the rest of this exercise, we will focus only on the numerical variables, for which it makes sense to replace every missing value with mean of the column. Those are `age` and `hours-per-week` columns.

###  Step 3: Create 'Dummy' Variables for Missing Values

No method of imputing missing values is perfect, and for this reason it makes sense to keep track of which values we artificially created. 

The code cell below looks at the the values in columns `age` and `hours-per-week` and stores the corresponding `True`/`False` values (True if the value is missing and False if the value is present) in new columns `age_na` and `hours-per-week_na`. Run the cell and inspect the new columns.

In [ ]:
df['age_na'] = df['age'].isnull()
df['hours-per-week_na'] = df['hours-per-week'].isnull()
df.head()

### Step 4: Fill the Missing Values Using Pandas `fillna()` Method

The Pandas `fillna()` method is used to "fill in" missing values in a Series or DataFrame object. Consult the online [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.fillna.html) for more information about how to use the `fillna()` method.
The code cell below uses `fillna()` to fill in values for the missing values in the `age` column.
It fills in the missing values with the mean value of all of the existing values in the that column. It uses the Pandas `mean()` method to compute the replacement values. 
For more information about `mean()`, consult the online [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.mean.html).

Tip: when working with `fillna()`, make sure that you do not just create a copy object with the filled values, but change the original values of the `df` object by specifying the `inplace = True` parameter value.

First inspect some of the columns that contain missing values.

In [ ]:
df.loc[df['age'].isnull()]


In [ ]:
# look at one row that contains a missing value for age
print("Row 654:  " + str(df['age'][654]))

# compute mean for all non null age values
mean_ages=df['age'].mean()
print("mean value for all age columns: " + str(mean_ages))

# fill all missing values with the mean
df['age'].fillna(value=mean_ages, inplace=True)

# look at one of the rows that contained a missing value for age. 
# It should now contain the mean
print("Row 654:  " + str(df['age'][654]))



In the code cell below, do the same for the `hours-per-week` column.

1. Compute the mean value of the `hours-per-week` column and save the result to variable `mean_hours`
2. Use `fillna` to change the values of the missing columns to `mean_hours`.

### Graded Cell

The cell below will be graded. Remove the line "raise NotImplementedError()" before writing your code.

In [ ]:
mean_hours = df['hours-per-week'].mean()
mean_hours = df['hours-per-week'].fillna(value = mean_hours, inplace = True)

### Self-Check

Run the cell below to test the correctness of your code above before submitting for grading. Do not add code or delete code in the cell.

In [ ]:
# Run this self-test cell to check your code; 
# do not add code or delete code in this cell
from jn import testFillNa

try:
    p, err = testFillNa(df)
    print(err)
except Exception as e:
    print("Error!\n" + str(e))
    


Check if we successfully converted all missing values to the mean value. Display the sum of missing values for the `age` column. 

In [ ]:
np.sum(df['age'].isnull(), axis = 0)

In the code cell below, do the same for the `hours-per-week` column. Save the result to variable `sum_hours`.

### Graded Cell

The cell below will be graded. Remove the line "raise NotImplementedError()" before writing your code.

In [ ]:
sum_hours = np.sum(df['hours-per-week'].isnull(), axis = 0)

### Self-Check

Run the cell below to test the correctness of your code above before submitting for grading. Do not add code or delete code in the cell.

In [ ]:
# Run this self-test cell to check your code; 
# do not add code or delete code in this cell
from jn import testSumHours

try:
    p, err = testSumHours(df, sum_hours)
    print(err)
except Exception as e:
    print("Error!\n" + str(e))
    
